In [1]:
import quickage

SyntaxError: invalid syntax (core.py, line 238)

In [ ]:
folder='/users/caganze/research/j1624/data/'
filename=folder+'/example_quickage.jpeg'

In [ ]:
sourc={'ra':46.0684208, \
        'dec': -32.203782,\
        'pmra':(-25.809, 0.159),
        'pmdec':(-185.783, 0.122),
        'distance':(16.9465, 0.0356),
        'rv': (0.23, 0.08)}
metal=(0.29, 0.01)

In [ ]:
res=quickage.estimate_age(sourc, metal)
res['median_age'], res['std_age']

In [ ]:
res=quickage.estimate_age(sourc, metal, nsigma=1, plot=True, file_plot=filename)


In [ ]:
res['median_age'], res['std_age']